En este notebook vamos a aplicar EM (Expectation–Maximization) a los datos de Novadent para construir un modelo de atribución multitouch (MTA).

In [1]:
import pandas as pd

In [2]:
# Cargar contribuciones en data/NOVADENT/processed/altas_contributions.parquet y data/NOVADENT/processed/contactos_contributions.parquet
altas_contributions = pd.read_parquet('data/NOVADENT/processed/altas_contributions.parquet')
contactos_contributions = pd.read_parquet('data/NOVADENT/processed/contactos_contributions.parquet')

In [7]:
contactos_contributions.head()

,date,radio_contribution,tv_contribution,baseline,total_predicted
0,2018-01-01,2.977236,0.0,182.788065,185.765302
1,2018-01-02,2.084065,0.0,696.673222,698.757288
2,2018-01-03,1.458846,0.0,621.342196,622.801042
3,2018-01-04,1.021192,0.0,506.115505,507.136697
4,2018-01-05,3.940174,0.0,542.511513,546.451686


In [6]:
# Cargar data/NOVADENT/processed/pases_radio_processed.parquet y data/NOVADENT/processed/pases_tv_processed.parquet
pases_radio = pd.read_parquet('data/NOVADENT/processed/pases_radio_processed.parquet')
pases_tv = pd.read_parquet('data/NOVADENT/processed/pases_tv_processed.parquet')

In [8]:
pases_radio.head()

,id,pase,campana,cadena,ambitos,fecha,dia,hora,dur,pb2,...,month,dayofweek,week,franja,grps20,coste_por_grp20,coste_por_grp,coste_por_contacto,contactos_por_segundo,grps_por_segundo
0,8221,EL PIRATA Y SU BANDA,NOVADENT,ROCK FM CADENA,NACIONAL,2018-02-12,LUNES,08:11:08,20,1,...,2,0,7,mañana,0.4,525.0,525.0,0.002861,3670.062,0.02
1,8222,MARTA VAZQUEZ,NOVADENT,ROCK FM CADENA,NACIONAL,2018-02-12,LUNES,11:05:17,20,1,...,2,0,7,mañana,0.4,469.0,469.0,0.002556,3670.062,0.02
2,8223,MARTA VAZQUEZ,NOVADENT,ROCK FM CADENA,NACIONAL,2018-02-12,LUNES,13:27:33,20,1,...,2,0,7,mediodía,0.3,625.333333,625.333333,0.003408,2752.5465,0.015
3,8224,RAUL CARNICERO,NOVADENT,ROCK FM CADENA,NACIONAL,2018-02-12,LUNES,18:01:30,20,1,...,2,0,7,mediodía,0.2,805.0,805.0,0.004387,1835.031,0.01
4,8225,RODRI CONTRERAS,NOVADENT,ROCK FM CADENA,NACIONAL,2018-02-12,LUNES,20:26:43,20,1,...,2,0,7,tarde,0.1,1610.0,1610.0,0.008774,917.5155,0.005


In [9]:
pases_tv.head()

,id,campana,mes,tipo_accion,grupo,soporte,fecha,dia,hora_real,dur,...,emision_ts,year,month_num,dayofweek,week,franja_calc,grps20,coste_por_grp20,coste_por_grp,grps_por_segundo
0,1007,NOVADENT,MAR,SPOT,A3COB,ATRES COBERTURA,2018-03-01,Jueves,11:48:58,20,...,2018-03-01 11:48:58,2018,3,3,9,mañana,1.9,441.0,441.0,0.095
1,1008,NOVADENT,MAR,SPOT,A3COB,ATRES COBERTURA,2018-03-01,Jueves,16:02:48,10,...,2018-03-01 16:02:48,2018,3,3,9,mediodía,5.12,308.7,617.4,0.256
2,1009,NOVADENT,MAR,SPOT,A3COB,ATRES COBERTURA,2018-03-01,Jueves,18:37:41,20,...,2018-03-01 18:37:41,2018,3,3,9,mediodía,6.08,441.0,441.0,0.304
3,1010,NOVADENT,MAR,SPOT,A3COB,ATRES COBERTURA,2018-03-01,Jueves,21:39:11,20,...,2018-03-01 21:39:11,2018,3,3,9,tarde,7.22,441.0,441.0,0.361
4,1011,NOVADENT,MAR,SPOT,A3COB,ATRES COBERTURA,2018-03-04,Domingo,22:01:36,10,...,2018-03-04 22:01:36,2018,3,6,9,tarde,5.43,308.7,617.4,0.2715


# Multitouch Attribution usando EM



## Propuesta de modelado MTA (EM) para Novadent: repartir contribuciones MMM (radio/tv) a pases

> Basado en la idea **HCP–Brick** del notebook end-to-end: tenemos un **total observado agregado** por unidad (aquí: canal×día) y queremos repartirlo a unidades más finas (aquí: *pases/spots*) usando **exposición** y una **productividad latente** aprendida con EM. Y, para ser coherentes con MMM, usamos (o imitamos) las transformaciones de **adstock + saturación** del notebook MMM→MTA.

---
### 1) Qué consideramos observado vs latente
**Observado (de `contactos_contributions`)**
- Para cada día $t$:
  - $C_{radio,t}$ = `radio_contribution`
  - $C_{tv,t}$ = `tv_contribution`
  - (opcional para contexto) `baseline`, `total_predicted`

> Interpretación: $C_{m,t}$ es el “presupuesto de contribución” del canal $m$ que debemos **repartir** hacia pases concretos de ese canal en ese día (o en una ventana temporal si hay carryover).

**Disponibles como evidencia/exposición (de `pases_radio`, `pases_tv`)**
- Registro a nivel pase/spot (lo que sea que represente una fila): fecha/hora, cadena/emisora, franja, duración, GRPs, coste, etc.
- Denotemos cada pase como $k$ dentro de un canal $m\in\{radio,tv\}$.

**Latente (lo que aprende EM)**
- $z_{m,k,t}\ge 0$: contribución del canal $m$ en el día $t$ que atribuimos al pase $k$.
- Restricción dura (debe cumplirse por construcción):
  $$\sum_{k\in\mathcal{K}_m} z_{m,k,t} = C_{m,t}$$

---
### 2) Construcción de exposición $w_{m,k,t}$ (adstock + saturación)
La clave para que el reparto tenga sentido es construir una señal $w_{m,k,t}\ge 0$ que refleje **cuánta presión publicitaria** del pase $k$ está “activa” en el día $t$.

**Opción A (simple y robusta): asignación same-day**
- Si asumimos que el efecto se concentra el mismo día del pase:
  - $w_{m,k,t} =$ GRPs (o coste) del pase $k$ si ocurre en $t$, y 0 si no.
- Ventaja: fácil, muy interpretable.
- Limitación: ignora carryover.

**Opción B (recomendada): carryover con adstock (coherente con MMM)**
- Cada pase crea un impulso de presión que decae en días posteriores.
- Para cada pase $k$ ocurrido en $\tau$, definimos contribución de exposición al día $t\ge \tau$ con decaimiento geométrico:
  $$w_{m,k,t}^{ad} = x_{m,k,\tau}\cdot \delta_m^{(t-\tau)}$$
  donde $x_{m,k,\tau}$ es la intensidad del pase (GRPs/coste/duración ponderada) y $\delta_m\in(0,1)$ es el decay (por canal).
- Si además quieres saturación tipo Hill (del notebook MMM):
  $$w_{m,k,t} = \text{Hill}(w_{m,k,t}^{ad};\ \alpha_m,\ \gamma_m) = \frac{(w_{m,k,t}^{ad})^{\alpha_m}}{(w_{m,k,t}^{ad})^{\alpha_m}+\gamma_m^{\alpha_m}}$$
- Práctica: puedes fijar $(\delta_m,\alpha_m,\gamma_m)$ a valores del MMM (si los tienes) o hacer un pequeño grid-search y quedarte con los que mejor estabilicen el reparto (sin sobreajustar).

---
### 3) Modelo generativo (Poisson) + EM (análogamente a HCP–Brick)
Tratamos la contribución $C_{m,t}$ como “unidades” a repartir (no tiene por qué ser entera; en la práctica funciona igual con EM por proporciones).

**Intensidad por pase**
- Cada pase tiene una productividad positiva $\lambda_{m,k}>0$ (constante en el tiempo) que captura “calidad media” del pase (emisora/franja/formato…).
- La media esperada de contribución del pase $k$ en día $t$ es:
  $$\mu_{m,k,t} = \lambda_{m,k}\, w_{m,k,t}$$

**E-step (reparto dentro de cada $m,t$)**
- Para cada canal $m$ y día $t$, calculamos pesos:
  $$p_{m,k,t} = \frac{\lambda_{m,k} w_{m,k,t}}{\sum_{k'} \lambda_{m,k'} w_{m,k',t} + 10^{-12}}$$
- Y atribuimos en esperanza:
  $$\hat z_{m,k,t} = C_{m,t}\cdot p_{m,k,t}$$

**M-step (actualización de productividades)**
- Sin covariables:
  $$\lambda_{m,k} \leftarrow \frac{\sum_t \hat z_{m,k,t}}{\sum_t w_{m,k,t} + 10^{-12}}$$
- Con covariables (recomendado si $K$ grande):
  - Definir features $X_{m,k}$ (emisora, franja, duración, región, etc.)
  - Parametrizar $\lambda_{m,k}=\exp(X_{m,k}\beta_m)$
  - Ajustar un **Poisson GLM con offset** $\log(\tilde w_{m,k})$ y ridge (IRLS + regularización), exactamente como en el notebook EM end-to-end.

---
### 4) Cómo aterrizarlo a tus 3 dataframes (pipeline)
1) **Extraer contribuciones diarias por canal**
- Crear un panel por día $t$ con `contactos_contributions[['date','radio_contribution','tv_contribution']]` y renombrar a $C_{radio,t}$, $C_{tv,t}$.

2) **Definir unidad de atribución $k$ por canal**
- Radio: (emisora × franja × programa) o (emisora × franja), según granularidad y estabilidad.
- TV: (cadena × franja × programa) o (cadena × bloque).
- La unidad debe equilibrar: suficiente señal (exposición no cero a menudo) + utilidad de negocio.

3) **Construir $w_{m,k,t}$ desde `pases_radio` / `pases_tv`**
- Elegir variable de intensidad $x$ (GRPs si existe; si no, coste; si no, duración ponderada).
- Agregar a nivel (k, fecha) y luego expandir a días posteriores con adstock (opción B).
- Aplicar saturación Hill si quieres replicar el MMM (opcional pero recomendable si el MMM la usa).

4) **Correr EM por canal**
- Ejecutar EM separado para radio y TV (porque $C_{radio,t}$ no debe repartirse a TV y viceversa).
- Inicialización simple: $\lambda_{m,k}=1$ o proporcional a $\sum_t w_{m,k,t}$.
- Parada: cambio relativo en $\lambda$ o en log-likelihood (si lo computas).

5) **Outputs útiles**
- Tabla a nivel pase/cluster $k$: contribución total $\sum_t \hat z_{m,k,t}$
- Tabla diaria: $\hat z_{m,k,t}$ para análisis temporal
- Agregaciones de negocio: por emisora/cadena, por franja, por programa, por región (si existe).

---
### 5) Checks y debugging (importante en EM)
- **Conservación**: para todo $m,t$, debe cumplirse $\sum_k \hat z_{m,k,t}=C_{m,t}$ (error numérico ~ $10^{-9}$).
- **Denominadores**: si $\sum_k \lambda_{m,k} w_{m,k,t}=0$ en algún día, ese día no tiene pases o la intensidad está a cero; usar smoothing (p.ej. $w\leftarrow w+\epsilon$) o excluir ese $t$ del reparto de ese canal.
- **Colinealidad / sobregranularidad**: si $K$ es enorme y muchos $k$ casi nunca aparecen, agrupar unidades o usar GLM con ridge.
- **Robustez**: comparar reparto con una baseline determinista: $\hat z_{m,k,t}=C_{m,t}\cdot w_{m,k,t}/\sum_{k'}w_{m,k',t}$. EM debería mejorar estabilidad/consistencia si hay heterogeneidad real.

---
### 6) Qué obtienes (interpretación)
- Un **MTA “coherente con MMM”**: no cambia el total por canal/día (respeta `radio_contribution` / `tv_contribution`), pero lo desagrega a pases con un criterio estadístico.
- $\lambda_{m,k}$ se interpreta como **productividad relativa** del cluster/pase dentro del canal, una vez controlada la exposición adstock+saturada.

Si quieres, en la siguiente celda puedo proponerte un esqueleto de código (funciones `build_exposure_w`, `em_deaggregate_channel`) reutilizando exactamente la estructura vectorizada del notebook EM end-to-end.

In [17]:
import numpy as np
import pandas as pd

def _pick_first_col(df: pd.DataFrame, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

def _ensure_daily_date(df: pd.DataFrame) -> pd.Series:
    """Return a normalized daily timestamp series for a df, trying common date column names."""
    col = _pick_first_col(df, ['date', 'fecha', 'Fecha', 'FECHA'])
    if col is None:
        raise ValueError("No encuentro columna de fecha en df (busqué: date/fecha)")
    s = pd.to_datetime(df[col], errors='coerce')
    if s.isna().any():
        # If there are parsing issues, surface them early
        bad = df.loc[s.isna(), [col]].head(5)
        raise ValueError(f"Fechas no parseables en columna '{col}'. Ejemplos:\n{bad}")
    return s.dt.normalize()

def geometric_adstock(values: np.ndarray, rate: float) -> np.ndarray:
    """Adstock geométrico (como en EDA): out[t] = x[t] + rate*out[t-1]."""
    out = np.empty_like(values, dtype=float)
    carry = 0.0
    for i, x in enumerate(values.astype(float)):
        carry = float(x) + float(rate) * carry
        out[i] = carry
    return out

def build_daily_exposure_with_adstock(
    pases_df: pd.DataFrame,
    channel_name: str,
    group_cols: list[str],
    intensity_col: str | None = None,
    adstock_rate: float = 0.7,
    eps: float = 1e-12,
    calendar: pd.DatetimeIndex | None = None,
 ) -> pd.DataFrame:
    """
    Devuelve un DataFrame long con columnas:
      date, <group_cols>, x (intensidad diaria), w (adstock)
    donde w se calcula por grupo con adstock geométrico.
    """
    df = pases_df.copy()
    df['date'] = _ensure_daily_date(df)

    if intensity_col is None:
        intensity_col = _pick_first_col(df, ['grps20', 'grps', 'grp', 'coste', 'contactos'])
    if intensity_col is None:
        raise ValueError(f"No encuentro columna intensidad en {channel_name}. Probé grps20/grps/grp/coste/contactos")
    if intensity_col not in df.columns:
        raise ValueError(f"intensity_col='{intensity_col}' no existe en {channel_name}")

    for c in group_cols:
        if c not in df.columns:
            raise ValueError(f"Falta columna '{c}' en {channel_name}. Columnas disponibles: {list(df.columns)[:30]}...")

    # Intensidad diaria por grupo
    daily = (
        df.groupby(group_cols + ['date'], as_index=False)
          .agg(x=(intensity_col, 'sum'))
          .sort_values(group_cols + ['date'])
    )
    daily['x'] = pd.to_numeric(daily['x'], errors='coerce').fillna(0.0)
    daily.loc[daily['x'] < 0, 'x'] = 0.0

    if calendar is None:
        calendar = pd.date_range(daily['date'].min(), daily['date'].max(), freq='D')
    cal = pd.DataFrame({'date': pd.DatetimeIndex(calendar)})

    # Completar días faltantes por grupo (x=0) para aplicar adstock correctamente
    groups = daily[group_cols].drop_duplicates()
    full = groups.merge(cal, how='cross')
    full = full.merge(daily, on=group_cols + ['date'], how='left')
    full['x'] = full['x'].fillna(0.0)

    # Adstock por grupo
    full = full.sort_values(group_cols + ['date']).reset_index(drop=True)
    w_list = []
    for _, g in full.groupby(group_cols, sort=False):
        w = geometric_adstock(g['x'].to_numpy(), rate=adstock_rate)
        w_list.append(pd.Series(w, index=g.index))
    full['w'] = pd.concat(w_list).sort_index().to_numpy()
    full['w'] = full['w'].clip(lower=eps)
    return full

def build_cluster_features(
    pases_df: pd.DataFrame,
    group_cols: list[str],
    numeric_cols: list[str] | None = None,
 ) -> pd.DataFrame:
    """
    Construye features por cluster (grupo) usando agregados numéricos + one-hot de categorías.
    Retorna df con group_cols + features numéricas + dummies.
    """
    df = pases_df.copy()
    for c in group_cols:
        if c not in df.columns:
            raise ValueError(f"Falta columna '{c}' para features")

    if numeric_cols is None:
        numeric_cols = [c for c in ['grps20', 'grps', 'grp', 'coste', 'contactos', 'dur'] if c in df.columns]
    # Agregados numéricos (medias)
    agg_dict = {c: 'mean' for c in numeric_cols}
    features_num = df.groupby(group_cols, as_index=False).agg(agg_dict) if agg_dict else df[group_cols].drop_duplicates()
    for c in numeric_cols:
        if c in features_num.columns:
            features_num[c] = pd.to_numeric(features_num[c], errors='coerce').fillna(0.0)
            features_num[c] = np.log1p(features_num[c])  # estabilizar escala

    # Dummies de categorías (group_cols)
    dummies = pd.get_dummies(df[group_cols], drop_first=False, dtype=float)
    dummies = pd.concat([df[group_cols], dummies], axis=1)
    dummies = dummies.groupby(group_cols, as_index=False).max()
    features = features_num.merge(dummies, on=group_cols, how='left')
    return features

def irls_poisson_ridge(X, y, offset=None, alpha=1e-2, max_iter=50, tol=1e-8):
    n, p = X.shape
    if offset is None:
        offset = np.zeros(n, dtype=float)
    beta = np.zeros(p, dtype=float)
    for _ in range(max_iter):
        eta = X @ beta + offset
        mu = np.exp(eta)
        W = mu
        z = eta + (y - mu) / (mu + 1e-12)
        sqrtW = np.sqrt(W)
        Xw = X * sqrtW[:, None]
        yw = (z - offset) * sqrtW
        A = Xw.T @ Xw + alpha * np.eye(p)
        rhs = Xw.T @ yw
        beta_new = np.linalg.solve(A, rhs)
        if np.linalg.norm(beta_new - beta) / (np.linalg.norm(beta) + 1e-12) < tol:
            beta = beta_new
            break
        beta = beta_new
    return beta

def em_deaggregate_contribution(
    contrib_df: pd.DataFrame,
    exposure_df: pd.DataFrame,
    channel_col_in_contrib: str,
    group_cols: list[str],
    cluster_features: pd.DataFrame | None = None,
    use_covariates: bool = True,
    alpha: float = 1e-2,
    max_iter: int = 200,
    tol: float = 1e-8,
    eps: float = 1e-12,
 ):
    """
    EM con covariables (por defecto):
      E-step: zhat_{k,t} = C_t * (lambda_k w_{k,t}) / sum_k(lambda_k w_{k,t})
      M-step: lambda_k = exp(X_k beta)  (Poisson GLM con offset log(w_sum))
    """
    cdf = contrib_df.copy()
    cdf['date'] = _ensure_daily_date(cdf)
    if channel_col_in_contrib not in cdf.columns:
        raise ValueError(f"No existe columna '{channel_col_in_contrib}' en contrib_df")
    cdf['C'] = pd.to_numeric(cdf[channel_col_in_contrib], errors='coerce').fillna(0.0)
    cdf.loc[cdf['C'] < 0, 'C'] = 0.0

    edf = exposure_df.copy()
    for c in ['date', 'w'] + group_cols:
        if c not in edf.columns:
            raise ValueError(f"exposure_df debe contener '{c}'")
    edf['date'] = pd.to_datetime(edf['date']).dt.normalize()
    edf['w'] = pd.to_numeric(edf['w'], errors='coerce').fillna(0.0).clip(lower=eps)

    dates = np.sort(cdf['date'].unique())
    edf = edf.loc[edf['date'].isin(dates)].copy()
    if edf.empty:
        raise ValueError("No hay exposición solapando con fechas de contribuciones")

    # Matriz W: T x K
    k_df = edf[group_cols].drop_duplicates().reset_index(drop=True)
    k_df['k'] = np.arange(len(k_df), dtype=int)
    edf = edf.merge(k_df, on=group_cols, how='left')
    date_map = {d: i for i, d in enumerate(dates)}
    edf['t'] = edf['date'].map(date_map).astype(int)
    T = len(dates)
    K = len(k_df)
    W = np.zeros((T, K), dtype=float)
    W[edf['t'].to_numpy(), edf['k'].to_numpy()] = edf['w'].to_numpy()

    C = cdf.set_index('date').reindex(dates)['C'].to_numpy(dtype=float)
    C = np.nan_to_num(C, nan=0.0)
    C = np.clip(C, 0.0, None)

    # Features por cluster
    if use_covariates:
        if cluster_features is None:
            raise ValueError("use_covariates=True pero no se pasó cluster_features")
        feat = cluster_features.copy()
        feat = k_df.merge(feat, on=group_cols, how='left')
        feat = feat.drop(columns=['k'] + group_cols)
        feat = feat.fillna(0.0)
        # convertir cualquier residual a numérico
        feat = feat.apply(pd.to_numeric, errors='coerce').fillna(0.0)
        X = feat.to_numpy(dtype=float)
        # estandarizar
        col_std = X.std(axis=0)
        col_std[col_std == 0] = 1.0
        X = (X - X.mean(axis=0)) / col_std
    else:
        X = None

    w_sum = W.sum(axis=0)
    lam = np.where(w_sum > 0, 1.0, 0.0)
    lam = np.clip(lam, eps, None)
    beta = None

    history = []
    for _ in range(max_iter):
        mu = W * lam[None, :]
        denom = mu.sum(axis=1) + eps
        Z = (C[:, None] * (mu / denom[:, None]))
        if use_covariates:
            y_tilde = Z.sum(axis=0)
            offset = np.log(w_sum + eps)
            beta = irls_poisson_ridge(X, y_tilde, offset=offset, alpha=alpha, max_iter=60, tol=1e-8)
            lam_new = np.exp(X @ beta)
        else:
            lam_new = Z.sum(axis=0) / (w_sum + eps)
        lam_new = np.clip(lam_new, eps, None)
        rel = float(np.linalg.norm(lam_new - lam) / (np.linalg.norm(lam) + eps))
        history.append(rel)
        lam = lam_new
        if rel < tol:
            break

    # Salida long
    zhat_long = pd.DataFrame(Z, columns=[f'k_{i}' for i in range(K)])
    zhat_long['date'] = dates
    zhat_long = zhat_long.melt(id_vars=['date'], var_name='k', value_name='zhat')
    zhat_long['k'] = zhat_long['k'].str.replace('k_', '', regex=False).astype(int)
    zhat_long = zhat_long.merge(k_df, on='k', how='left', suffixes=('', '_drop'))
    if 'k_drop' in zhat_long.columns:
        zhat_long = zhat_long.drop(columns=['k_drop'])
    w_long = pd.DataFrame(W, columns=np.arange(K))
    w_long['date'] = dates
    w_long = w_long.melt(id_vars=['date'], var_name='k', value_name='w')
    w_long['k'] = w_long['k'].astype(int)
    zhat_long = zhat_long.merge(w_long, on=['date', 'k'], how='left')
    zhat_long['lambda'] = zhat_long['k'].map({i: lam[i] for i in range(K)})
    zhat_long['channel'] = channel_col_in_contrib

    check = (
        zhat_long.groupby('date', as_index=False)['zhat'].sum()
                 .rename(columns={'zhat': 'sum_zhat'})
                 .merge(cdf[['date','C']], on='date', how='left')
    )
    check['abs_diff'] = (check['sum_zhat'] - check['C']).abs()
    diagnostics = {
        'n_iter': len(history),
        'last_rel_change': history[-1] if history else None,
        'max_abs_conservation_error': float(check['abs_diff'].max()),
        'conservation_table': check,
    }
    lambda_df = k_df.drop(columns=['k']).copy()
    lambda_df['lambda'] = lam
    return zhat_long, lambda_df, diagnostics, beta

# ----------------------
# Ejecución: RADIO y TV
# ----------------------
contrib = contactos_contributions.copy()
contrib['date'] = pd.to_datetime(contrib['date']).dt.normalize()

# Definir clusters (ajusta granularidad si lo necesitas)
radio_group_cols = [c for c in ['cadena', 'franja'] if c in pases_radio.columns]
if not radio_group_cols:
    radio_group_cols = [c for c in ['cadena'] if c in pases_radio.columns]
tv_group_cols = [c for c in ['cadena', 'franja_calc'] if c in pases_tv.columns]
if len(tv_group_cols) < 2:
    tv_group_cols = [c for c in ['cadena', 'franja'] if c in pases_tv.columns]
if not tv_group_cols:
    tv_group_cols = [c for c in ['cadena'] if c in pases_tv.columns]

calendar = pd.date_range(contrib['date'].min(), contrib['date'].max(), freq='D')

# Adstock como en EDA (sin saturación)
radio_exposure = build_daily_exposure_with_adstock(
    pases_radio,
    channel_name='radio',
    group_cols=radio_group_cols,
    intensity_col=None,
    adstock_rate=0.7,
    calendar=calendar,
 )
tv_exposure = build_daily_exposure_with_adstock(
    pases_tv,
    channel_name='tv',
    group_cols=tv_group_cols,
    intensity_col=None,
    adstock_rate=0.5,
    calendar=calendar,
 )

# Features por cluster (covariables)
radio_features = build_cluster_features(pases_radio, radio_group_cols)
tv_features = build_cluster_features(pases_tv, tv_group_cols)

# EM: repartir contribuciones MMM->clusters (con covariables)
radio_zhat, radio_lambda, radio_diag, radio_beta = em_deaggregate_contribution(
    contrib_df=contrib,
    exposure_df=radio_exposure,
    channel_col_in_contrib='radio_contribution',
    group_cols=radio_group_cols,
    cluster_features=radio_features,
    use_covariates=True,
    alpha=1e-2,
 )
tv_zhat, tv_lambda, tv_diag, tv_beta = em_deaggregate_contribution(
    contrib_df=contrib,
    exposure_df=tv_exposure,
    channel_col_in_contrib='tv_contribution',
    group_cols=tv_group_cols,
    cluster_features=tv_features,
    use_covariates=True,
    alpha=1e-2,
 )

print('RADIO max conservation error:', radio_diag['max_abs_conservation_error'])
print('TV    max conservation error:', tv_diag['max_abs_conservation_error'])
display(radio_lambda.sort_values('lambda', ascending=False).head(10))
display(tv_lambda.sort_values('lambda', ascending=False).head(10))

RADIO max conservation error: 1.0231815394945443e-12
TV    max conservation error: 1.0373923942097463e-12


,cadena,franja,lambda
19,M 80 CADENA,mañana,1.687304
9,COPE CADENA,noche,1.667221
34,SER BARCELONA,mediodía,1.622945
28,ROCK FM CADENA,mañana,1.600809
35,SER BARCELONA,noche,1.584690
15,LOS 40 PRINCIPALES CADENA,mañana,1.575507
7,COPE CADENA,mañana,1.569658
5,COPE BARCELONA,noche,1.568251
0,CADENA 100 CADENA,mañana,1.566536
37,SER CADENA,mediodía,1.563118


,cadena,franja_calc,lambda
10,FORTA,mañana,2.438231
20,NSF,mañana,1.813985
13,FORTA,prime,1.442326
22,NSF,noche,1.441458
2,A3COB,noche,1.260685
12,FORTA,noche,1.224951
23,NSF,prime,1.099835
14,FORTA,tarde,1.072199
21,NSF,mediodía,1.019048
24,NSF,tarde,1.008479


In [18]:
radio_lambda

,cadena,franja,lambda
0,CADENA 100 CADENA,mañana,1.566536
1,CADENA 100 CADENA,mediodía,1.555589
2,CADENA 100 CADENA,noche,1.375725
3,COPE BARCELONA,mañana,1.522789
4,COPE BARCELONA,mediodía,1.424567
5,COPE BARCELONA,noche,1.568251
6,COPE BARCELONA,tarde,1.506875
7,COPE CADENA,mañana,1.569658
8,COPE CADENA,mediodía,1.440713
9,COPE CADENA,noche,1.667221


In [22]:
radio_lambda.groupby(['cadena']).agg(total_lambda=('lambda', 'sum')).sort_values('total_lambda', ascending = False).reset_index()

,cadena,total_lambda
0,COPE BARCELONA,6.022482
1,ROCK FM CADENA,5.883495
2,DIAL CADENA,5.809969
3,SER BARCELONA,4.757276
4,COPE CADENA,4.677593
5,LOS 40 PRINCIPALES CADENA,4.663215
6,SER CADENA,4.640090
7,CADENA 100 CADENA,4.497851
8,M 80 CADENA,4.478704
9,RADIOLE,2.967376


In [23]:
radio_lambda.groupby(['franja']).agg(total_lambda=('lambda', 'sum')).sort_values('total_lambda', ascending = False).reset_index()

,franja,total_lambda
0,mediodía,18.661909
1,mañana,16.728622
2,noche,9.977895
3,tarde,8.159645
4,prime,0.374382


In [19]:
tv_lambda

,cadena,franja_calc,lambda
0,A3COB,mañana,0.913150
1,A3COB,mediodía,0.842767
2,A3COB,noche,1.260685
3,A3COB,prime,1.005932
4,A3COB,tarde,0.819200
5,CUATRO,mañana,0.984501
6,CUATRO,mediodía,0.875747
7,CUATRO,noche,0.893953
8,CUATRO,prime,0.818927
9,CUATRO,tarde,0.835112
